# Building Chatbot app using Mistral 7b

**Mistral**

- Es una red neuronal del tipo "mezcla dispersa de expertos" (MoE, mixture of experts, por sus siglas en inglés).

- Es un modelo "solo decodificador", lo que significa que solo utiliza la información del contexto para generar salidas.

- Está preentrenado con datos multilingües, lo que le permite procesar y generar texto en varios idiomas.

- Utiliza un tamaño de contexto de 32k tokens, lo que le permite tener en cuenta una gran cantidad de información previa al generar la salida.

**Mistral 7b**

- Se refiere a una versión específica de Mistral con 7.3 mil millones de parámetros.

- Este tamaño indica la complejidad del modelo y su capacidad para aprender patrones complejos en los datos.

En resumen, Mistral 7b es un modelo de lenguaje multilingüe y complejo que utiliza la arquitectura MoE para procesar y generar texto de manera eficiente.

**huggingface_hub library**

Utilizaremos la libreria hugingface_hub para una interacción más fácil con el modelo. Ofrece las siguientes funciones:

- **Inference Client and Async Inference Clien**: Ejecuta el modelo de forma síncrona o asíncrona.

- **Token streaming**: Carga solo los tokens necesarios (eficiente para mensajes largos).

- **Detalles de la generación**: Accede a información como las probabilidades de los tokens y la detección del final de la oración.

**Formato de los mensajes, modelo Instruct:**

- Para beneficiarse del ajuste fino del modelo Instruct, sus mensajes necesitan un formato especial.

- Encierre sus instrucciones entre tokens [INST] y [\INST].

- La primera instrucción debe comenzar con un ID de "inicio de oración".

- Las instrucciones posteriores no lo necesitan.

- El modelo dejará de generar texto cuando encuentre el ID de token "fin de oración".







In [14]:
%pip install huggingface_hub gradio

from huggingface_hub import InferenceClient

client = InferenceClient(
    "mistralai/Mistral-7B-Instruct-v0.1"
)

prompt = """<s>[INST] ¿Cuál es tu ingrediente favorito?  [/INST]</s>
"""

res = client.text_generation(prompt, max_new_tokens=95)
print(res)

res = client.text_generation(prompt, max_new_tokens=35, stream=True, details=True, return_full_text=False)
for r in res: # this is a generator
  # print the token for example
  print(r)
  continue


[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.

Mi ingrediente favorito es el agua. Es un ingrediente esencial para la vida y es muy versátil en la cocina. Puedes usarlo para hacer todo tipo de platos, desde bebidas refrescantes como agua fresca y jugo hasta comidas como arroz con pollo y sopa de tomate. También es muy bueno para la salud y ayuda a mantener tu cuer
TextGenerationStreamResponse(token=Token(id=13, text='\n', logprob=-0.14111328, special=False), generated_text=None, details=None)
TextGenerationStreamResponse(token=Token(id=28755, text='M', logprob=-1.8740234, special=False), generated_text=None, details=None)
TextGenerationStreamResponse(token=Token(id=28710, text='i', logprob=-0.17626953, special=False), generated_text=None, details=None)
TextGenerationStreamResponse(token=Token(id=10887, text=' ingred', logprob=-0.014427185, special=False), generated_text=None, details=None)
TextGenerationStreamResponse(token=Token(id=8783, text='iente', logprob=-0.00

Con el streaming de tokens, el servidor devuelve los tokens a medida que se generan. Esto significa que no tienes que esperar a que se genere toda la respuesta antes de empezar a procesarla. La opción `return_full_text` se utiliza para controlar si la salida del modelo de lenguaje debe incluir el texto original de entrada o no.

El parámetro `max_new_tokens` indica el número máximo de tokens nuevos que se debe generar. Al fijar `stream=True`, indicas que quieres recibir la respuesta en trozos en lugar de descargar todo el cuerpo de la respuesta de una sola vez. Esto es útil para respuestas largas o cuando es importante recibir información rápidamente.

**Estructura multi-instrucciones:**

¡Ahora vayamos un poco más allá y probemos una estructura multi-instrucciones! En este enfoque, puedes proporcionar múltiples instrucciones consecutivas al modelo, permitiéndole seguir un flujo de tareas o preguntas complejas. Recuerda que para aprovechar el ajuste fino del modelo Instruct, debes enmarcar cada instrucción utilizando los tokens `[INST]` y `[\INST]`.

**Ejemplo**

"[INST]Explica qué es la inteligencia artificial como un pirata.[INST] [INST]Ahora, dame un ejemplo de cómo la IA se usa en la vida cotidiana.[INST]"

Con este prompt, le pides al modelo que primero explique la IA como un pirata y luego proporcione un ejemplo de su uso cotidiano.

El uso de tokens de streaming y una estructura multi-instrucciones te permite aprovechar la potencia de Mistral 7b para crear chatbots interactivos y dinámicos. No dudes en experimentar con diferentes enfoques y ver qué mejor se adapta a tus necesidades.

In [11]:
def format_prompt(message, history):
  prompt = "<s>"
  for user_prompt, bot_response in history:
    prompt += f"[INST] {user_prompt} [/INST]"
    prompt += f" {bot_response}</s> "
  prompt += f"[INST] {message} [/INST]"
  return prompt

message = "¿Y qué piensa al respecto?"
history = [["¿Cuál es tu ingrediente favorito?", "Mi ingrediente favorito es el ketchup. Es versátil, sabroso y combina bien con una gran variedad de alimentos.."]]

In [12]:
print(format_prompt(message, history))

<s>[INST] ¿Cuál es tu ingrediente favorito? [/INST] Mi ingrediente favorito es el ketchup. Es versátil, sabroso y combina bien con una gran variedad de alimentos..</s> [INST] ¿Y qué piensa al respecto? [/INST]


## Chatbot using Gradio

Ahora vamos a crear una app usando Gradio que permita realizar varias interacciones consecutivas en una conversación, formatear las entradas con la estructura correcta, permitir al usuario modificar los parámetros y detener la generación.

Gradio es una biblioteca de código abierto de Python que te permite crear interfaces interactivas basadas en web para modelos de aprendizaje automático de forma rápida y sencilla. 

**Nucleus sampling también llamado top-p sampling:**

Este método es una técnica de decodificación utilizada en el NLP para generar texto. Selecciona el conjunto más pequeño posible de palabras cuya probabilidad acumulada exceda la probabilidad p. El número de palabras en el conjunto puede aumentar o disminuir dinámicamente según la distribución de probabilidad de la siguiente palabra. Este método es útil para generar texto con una longitud específica o cuando la salida necesita ser diversa.

In [13]:
%pip install gradio

import gradio as gr

# Esta función genera la respuesta del chatbot teniendo en cuenta el mensaje actual, 
# el historial de conversación, y varios parámetros configurables.
def generate(
    prompt, history, temperature=0.9, max_new_tokens=256, top_p=0.95, repetition_penalty=1.0,
):
    temperature = float(temperature)
    if temperature < 1e-2:
        temperature = 1e-2
    top_p = float(top_p)

    generate_kwargs = dict(
        temperature=temperature,
        max_new_tokens=max_new_tokens,
        top_p=top_p,
        repetition_penalty=repetition_penalty,
        do_sample=True,
        seed=42,
    )

    #La función format_prompt prepara el mensaje actual teniendo en cuenta el historial de conversación.
    formatted_prompt = format_prompt(prompt, history)

    #Se utiliza client.text_generation para generar la respuesta del chatbot de forma incremental, 
    # permitiendo procesarla o mostrarla mientras se crea.
    stream = client.text_generation(formatted_prompt, **generate_kwargs, stream=True, details=True, return_full_text=False)
    output = ""

    #Se construye la respuesta completa a partir de los tokens generados incrementalmente.
    for response in stream:
        output += response.token.text
        yield output
    return output

# Se definen sliders interactivos para modificar los parámetros
additional_inputs=[
    gr.Slider(
        label="Temperature",
        value=0.9,
        minimum=0.0,
        maximum=1.0,
        step=0.05,
        interactive=True,
        info="Higher values produce more diverse outputs",
    ),
    gr.Slider(
        label="Max new tokens",
        value=256,
        minimum=0,
        maximum=8192,
        step=64,
        interactive=True,
        info="The maximum numbers of new tokens",
    ),
    gr.Slider(
        label="Top-p (nucleus sampling)",
        value=0.90,
        minimum=0.0,
        maximum=1,
        step=0.05,
        interactive=True,
        info="Higher values sample more low-probability tokens",
    ),
    gr.Slider(
        label="Repetition penalty",
        value=1.2,
        minimum=1.0,
        maximum=2.0,
        step=0.05,
        interactive=True,
        info="Penalize repeated tokens",
    )
]

# Se crea la interfaz de la app
with gr.Blocks() as demo:
    gr.ChatInterface(
        generate,
        additional_inputs=additional_inputs,
    )

# Ejecución de la app
demo.queue().launch(debug=True)


[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.
Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


Keyboard interruption in main thread... closing server.
